# 💻 1.4 Scripting Animation Cơ Bản trong Roblox (Luau)

---

## Mục Tiêu
> Học cách viết script Luau để load, phát và điều khiển animation trong game Roblox.

---

# 📚 PHẦN 1: CÁC CLASS QUAN TRỌNG

---

## 1.1 Tổng quan Animation System

### Animation Pipeline trong Roblox

```
┌─────────────────────────────────────────────────────────────────────────┐
│                    ANIMATION PIPELINE                                    │
├─────────────────────────────────────────────────────────────────────────┤
│                                                                         │
│  ┌───────────────┐                                                      │
│  │   Animation   │  ← Bước 1: Tạo Animation object với AnimationId      │
│  │ (AnimationId) │                                                      │
│  └───────┬───────┘                                                      │
│          │                                                              │
│          │ LoadAnimation()                                              │
│          ▼                                                              │
│  ┌───────────────┐                                                      │
│  │   Animator    │  ← Bước 2: Animator load animation                   │
│  │  (on Humanoid)│                                                      │
│  └───────┬───────┘                                                      │
│          │                                                              │
│          │ Returns AnimationTrack                                       │
│          ▼                                                              │
│  ┌───────────────┐                                                      │
│  │AnimationTrack │  ← Bước 3: Điều khiển Play/Stop/Speed/Weight         │
│  │ (Playable)    │                                                      │
│  └───────┬───────┘                                                      │
│          │                                                              │
│          │ Play()                                                       │
│          ▼                                                              │
│  ┌───────────────┐                                                      │
│  │   Character   │  ← Bước 4: Animation phát trên character             │
│  │  (Animating)  │                                                      │
│  └───────────────┘                                                      │
│                                                                         │
└─────────────────────────────────────────────────────────────────────────┘
```

### 3 Class chính

| Class | Vai trò | Tạo từ đâu? |
|-------|---------|-------------|
| **Animation** | Chứa AnimationId, là "bản thiết kế" | `Instance.new("Animation")` |
| **Animator** | Load và quản lý các AnimationTrack | Có sẵn trong Humanoid |
| **AnimationTrack** | Animation đã load, có thể Play/Stop | `Animator:LoadAnimation()` |

## 1.2 Animation Class

### Animation là gì?
**Animation** là một Instance chứa tham chiếu đến animation đã publish trên Roblox thông qua `AnimationId`.

### Tạo Animation Object

```lua
-- Cách 1: Tạo bằng code
local animation = Instance.new("Animation")
animation.AnimationId = "rbxassetid://123456789"

-- Cách 2: Tạo trong Studio và reference
-- (Tạo Animation object trong ReplicatedStorage)
local animation = game.ReplicatedStorage:WaitForChild("WalkAnimation")
```

### Properties của Animation

```
┌─────────────────────────────────────────────────────────────────────────┐
│                    ANIMATION PROPERTIES                                  │
├─────────────────────────────────────────────────────────────────────────┤
│                                                                         │
│  Property         Type        Mô tả                                     │
│  ─────────────────────────────────────────────────────────────────────  │
│  AnimationId      string      ID của animation (rbxassetid://...)       │
│  Name             string      Tên của Instance                          │
│  Parent           Instance    Parent trong hierarchy                    │
│                                                                         │
└─────────────────────────────────────────────────────────────────────────┘
```

### Ví dụ chi tiết

```lua
-- Tạo và thiết lập Animation
local animation = Instance.new("Animation")
animation.Name = "WaveEmote"  -- Đặt tên dễ nhớ
animation.AnimationId = "rbxassetid://507770239"  -- Animation ID

-- Optional: Lưu vào một nơi để dùng lại
animation.Parent = game.ReplicatedStorage

print(animation.AnimationId)  -- Output: rbxassetid://507770239
```

### Lưu ý quan trọng

```
┌─────────────────────────────────────────────────────────────────────────┐
│  ⚠️ LƯU Ý VỀ ANIMATION ID                                               │
├─────────────────────────────────────────────────────────────────────────┤
│                                                                         │
│  ✅ ĐÚNG:                                                               │
│  animation.AnimationId = "rbxassetid://123456789"                       │
│                                                                         │
│  ❌ SAI:                                                                │
│  animation.AnimationId = "123456789"            -- Thiếu prefix        │
│  animation.AnimationId = "https://roblox.com/..." -- Không dùng URL    │
│  animation.AnimationId = "rbxasset://..."       -- Sai prefix          │
│                                                                         │
│  ⚠️ Animation phải có cùng Owner với Game!                              │
│                                                                         │
└─────────────────────────────────────────────────────────────────────────┘
```

## 1.3 Animator Class

### Animator là gì?
**Animator** là component nằm trong Humanoid, chịu trách nhiệm load animations và quản lý các AnimationTrack đang chạy.

### Vị trí của Animator

```
┌─────────────────────────────────────────────────────────────────────────┐
│                    ANIMATOR LOCATION                                     │
├─────────────────────────────────────────────────────────────────────────┤
│                                                                         │
│  Character (Model)                                                      │
│  ├── Humanoid                                                           │
│  │   └── Animator  ◄── ĐÂY LÀ ANIMATOR                                  │
│  ├── HumanoidRootPart                                                   │
│  ├── Head                                                               │
│  ├── UpperTorso                                                         │
│  └── ... (các parts khác)                                               │
│                                                                         │
└─────────────────────────────────────────────────────────────────────────┘
```

### Lấy Animator

```lua
-- Cách 1: Từ Humanoid
local humanoid = character:WaitForChild("Humanoid")
local animator = humanoid:WaitForChild("Animator")

-- Cách 2: Trực tiếp từ Character (khi chắc chắn có)
local animator = character:WaitForChild("Humanoid"):WaitForChild("Animator")

-- Cách 3: Tạo Animator nếu không có (hiếm khi cần)
local humanoid = character:WaitForChild("Humanoid")
local animator = humanoid:FindFirstChildOfClass("Animator")
if not animator then
    animator = Instance.new("Animator")
    animator.Parent = humanoid
end
```

### Methods của Animator

```
┌─────────────────────────────────────────────────────────────────────────┐
│                    ANIMATOR METHODS                                      │
├─────────────────────────────────────────────────────────────────────────┤
│                                                                         │
│  Method                       Return          Mô tả                     │
│  ─────────────────────────────────────────────────────────────────────  │
│  LoadAnimation(animation)     AnimationTrack  Load và trả về track     │
│  GetPlayingAnimationTracks()  {Track,...}     Danh sách tracks đang chạy│
│  StepAnimations(deltaTime)    void            Update animations thủ công│
│                                                                         │
└─────────────────────────────────────────────────────────────────────────┘
```

### Ví dụ sử dụng Animator

```lua
local humanoid = character:WaitForChild("Humanoid")
local animator = humanoid:WaitForChild("Animator")

-- Load animation
local animation = Instance.new("Animation")
animation.AnimationId = "rbxassetid://123456789"

local track = animator:LoadAnimation(animation)

-- Xem các animation đang chạy
local playingTracks = animator:GetPlayingAnimationTracks()
print("Đang chạy", #playingTracks, "animations")

for _, t in ipairs(playingTracks) do
    print("  -", t.Name)
end
```

## 1.4 AnimationTrack Class

### AnimationTrack là gì?
**AnimationTrack** là animation đã được load vào bộ nhớ, sẵn sàng để phát. Đây là object bạn tương tác nhiều nhất khi điều khiển animation.

### Tạo AnimationTrack

```lua
-- AnimationTrack được tạo bởi Animator:LoadAnimation()
local animation = Instance.new("Animation")
animation.AnimationId = "rbxassetid://123456789"

local track = animator:LoadAnimation(animation)  -- ← Đây là AnimationTrack
```

### Properties của AnimationTrack

```
┌─────────────────────────────────────────────────────────────────────────┐
│                    ANIMATIONTRACK PROPERTIES                             │
├─────────────────────────────────────────────────────────────────────────┤
│                                                                         │
│  READ-ONLY PROPERTIES:                                                  │
│  ─────────────────────────────────────────────────────────────────────  │
│  Animation       Animation    Animation object gốc                      │
│  IsPlaying       boolean      Có đang phát không?                       │
│  Length          number       Độ dài animation (giây)                   │
│  TimePosition    number       Vị trí hiện tại (giây)                    │
│  Speed           number       Tốc độ hiện tại                           │
│  WeightCurrent   number       Weight hiện tại (0-1)                     │
│  WeightTarget    number       Weight mục tiêu (0-1)                     │
│                                                                         │
│  READ-WRITE PROPERTIES:                                                 │
│  ─────────────────────────────────────────────────────────────────────  │
│  Looped          boolean      Có lặp lại không?                         │
│  Priority        Enum         Mức ưu tiên animation                     │
│                                                                         │
└─────────────────────────────────────────────────────────────────────────┘
```

### Methods của AnimationTrack

```
┌─────────────────────────────────────────────────────────────────────────┐
│                    ANIMATIONTRACK METHODS                                │
├─────────────────────────────────────────────────────────────────────────┤
│                                                                         │
│  CONTROL METHODS:                                                       │
│  ─────────────────────────────────────────────────────────────────────  │
│  Play(fadeTime, weight, speed)     Phát animation                       │
│  Stop(fadeTime)                    Dừng animation                       │
│  AdjustSpeed(speed)                Thay đổi tốc độ                      │
│  AdjustWeight(weight, fadeTime)    Thay đổi weight                      │
│                                                                         │
│  MARKER METHODS:                                                        │
│  ─────────────────────────────────────────────────────────────────────  │
│  GetMarkerReachedSignal(name)      Lấy signal khi đến marker            │
│  GetTimeOfKeyframe(name)           Lấy thời gian của keyframe           │
│                                                                         │
└─────────────────────────────────────────────────────────────────────────┘
```

### Events của AnimationTrack

```
┌─────────────────────────────────────────────────────────────────────────┐
│                    ANIMATIONTRACK EVENTS                                 │
├─────────────────────────────────────────────────────────────────────────┤
│                                                                         │
│  Event                    Khi nào fire?                                 │
│  ─────────────────────────────────────────────────────────────────────  │
│  Stopped                  Animation dừng (tự nhiên hoặc gọi Stop)       │
│  Ended                    Animation kết thúc (không loop)               │
│  DidLoop                  Animation vừa lặp lại                         │
│  KeyframeReached          Đến một keyframe có tên                       │
│                                                                         │
└─────────────────────────────────────────────────────────────────────────┘
```

---

# 🎯 PHẦN 2: KỸ NĂNG CẦN CÓ

---

## 2.1 Tạo Animation Object

### Cách 1: Tạo trong Script (Phổ biến nhất)

```lua
-- Tạo Animation object mới
local animation = Instance.new("Animation")
animation.AnimationId = "rbxassetid://123456789"

-- Optional: Đặt tên để debug dễ hơn
animation.Name = "WalkAnimation"
```

### Cách 2: Tạo trong Studio và Reference

```
┌─────────────────────────────────────────────────────────────────────────┐
│  BƯỚC THỰC HIỆN TRONG STUDIO:                                           │
├─────────────────────────────────────────────────────────────────────────┤
│                                                                         │
│  1. Trong Explorer, right-click ReplicatedStorage                       │
│  2. Insert Object → Animation                                           │
│  3. Đổi tên (ví dụ: "WalkAnim")                                         │
│  4. Trong Properties, set AnimationId                                   │
│                                                                         │
│  Kết quả trong Explorer:                                                │
│  ReplicatedStorage                                                      │
│  └── WalkAnim (Animation)                                               │
│      └── AnimationId = "rbxassetid://123456789"                         │
│                                                                         │
└─────────────────────────────────────────────────────────────────────────┘
```

```lua
-- Reference animation từ ReplicatedStorage
local ReplicatedStorage = game:GetService("ReplicatedStorage")
local walkAnim = ReplicatedStorage:WaitForChild("WalkAnim")

local track = animator:LoadAnimation(walkAnim)
```

### Cách 3: Sử dụng Module để quản lý

```lua
-- AnimationData.lua (ModuleScript trong ReplicatedStorage)
local AnimationData = {
    Walk = "rbxassetid://111111111",
    Run = "rbxassetid://222222222",
    Jump = "rbxassetid://333333333",
    Attack1 = "rbxassetid://444444444",
    Wave = "rbxassetid://555555555",
}

return AnimationData
```

```lua
-- Sử dụng trong LocalScript
local AnimationData = require(game.ReplicatedStorage.AnimationData)

local function createAnimation(animName)
    local anim = Instance.new("Animation")
    anim.AnimationId = AnimationData[animName]
    anim.Name = animName
    return anim
end

local walkAnim = createAnimation("Walk")
local attackAnim = createAnimation("Attack1")
```

## 2.2 Load Animation với Animator

### Cách cơ bản

```lua
-- Bước 1: Lấy Animator
local character = player.Character or player.CharacterAdded:Wait()
local humanoid = character:WaitForChild("Humanoid")
local animator = humanoid:WaitForChild("Animator")

-- Bước 2: Tạo Animation
local animation = Instance.new("Animation")
animation.AnimationId = "rbxassetid://123456789"

-- Bước 3: Load Animation
local track = animator:LoadAnimation(animation)

-- Bây giờ track sẵn sàng để Play!
```

### Load nhiều Animations

```lua
-- Tạo table lưu trữ các tracks
local animationTracks = {}

local animationIds = {
    Walk = "rbxassetid://111111111",
    Run = "rbxassetid://222222222",
    Jump = "rbxassetid://333333333",
    Attack1 = "rbxassetid://444444444",
}

-- Load tất cả animations
for name, id in pairs(animationIds) do
    local anim = Instance.new("Animation")
    anim.AnimationId = id
    anim.Name = name
    
    animationTracks[name] = animator:LoadAnimation(anim)
end

-- Sử dụng
animationTracks.Walk:Play()
animationTracks.Attack1:Play()
```

### Best Practice: Preload Animations

```
┌─────────────────────────────────────────────────────────────────────────┐
│                    PRELOAD ANIMATIONS                                    │
├─────────────────────────────────────────────────────────────────────────┤
│                                                                         │
│  TẠI SAO PRELOAD?                                                       │
│  • Animation cần thời gian để load từ server                            │
│  • Nếu Play ngay sau LoadAnimation, có thể bị delay                     │
│  • Preload giúp animation sẵn sàng ngay khi cần                         │
│                                                                         │
│  KHI NÀO PRELOAD?                                                       │
│  • Khi character spawn                                                  │
│  • Khi player equip weapon                                              │
│  • Ở loading screen                                                     │
│                                                                         │
└─────────────────────────────────────────────────────────────────────────┘
```

```lua
-- Preload animation khi character spawn
local function setupCharacter(character)
    local humanoid = character:WaitForChild("Humanoid")
    local animator = humanoid:WaitForChild("Animator")
    
    -- Preload tất cả animations
    local tracks = {}
    for name, id in pairs(animationIds) do
        local anim = Instance.new("Animation")
        anim.AnimationId = id
        tracks[name] = animator:LoadAnimation(anim)
    end
    
    return tracks
end

-- Khi character spawn
player.CharacterAdded:Connect(function(character)
    local tracks = setupCharacter(character)
    -- Lưu tracks để dùng sau
end)
```

## 2.3 Play và Stop Animation

### Play Animation

```lua
-- Cách đơn giản nhất
track:Play()

-- Với các parameters
track:Play(fadeTime, weight, speed)

-- Ví dụ:
track:Play(0.2)           -- Fade in 0.2 giây
track:Play(0.1, 1, 1.5)   -- Fade 0.1s, weight 1, speed 1.5x
```

### Parameters của Play()

```
┌─────────────────────────────────────────────────────────────────────────┐
│                    PLAY() PARAMETERS                                     │
├─────────────────────────────────────────────────────────────────────────┤
│                                                                         │
│  Parameter    Type      Default    Mô tả                                │
│  ─────────────────────────────────────────────────────────────────────  │
│  fadeTime     number    0.1        Thời gian transition (giây)          │
│  weight       number    1          Độ mạnh animation (0-1)              │
│  speed        number    1          Tốc độ phát (1 = bình thường)        │
│                                                                         │
│  VÍ DỤ:                                                                 │
│  track:Play()              -- Mặc định: fade 0.1s, weight 1, speed 1    │
│  track:Play(0)             -- Không fade, bắt đầu ngay                  │
│  track:Play(0.5)           -- Fade in mượt 0.5s                         │
│  track:Play(0.1, 0.5)      -- Weight 50% (blend với animation khác)     │
│  track:Play(0.1, 1, 2)     -- Tốc độ gấp đôi                            │
│  track:Play(0.1, 1, -1)    -- Phát ngược!                               │
│                                                                         │
└─────────────────────────────────────────────────────────────────────────┘
```

### Stop Animation

```lua
-- Dừng ngay lập tức
track:Stop()

-- Dừng với fade out
track:Stop(0.2)  -- Fade out 0.2 giây

-- Dừng tất cả animations
for _, t in ipairs(animator:GetPlayingAnimationTracks()) do
    t:Stop(0.1)
end
```

### Ví dụ thực tế: Toggle Animation

```lua
local isPlaying = false

local function toggleAnimation()
    if isPlaying then
        track:Stop(0.2)
        isPlaying = false
    else
        track:Play(0.2)
        isPlaying = true
    end
end

-- Hoặc dùng property IsPlaying
local function toggleAnimation()
    if track.IsPlaying then
        track:Stop(0.2)
    else
        track:Play(0.2)
    end
end
```

## 2.4 Thiết lập Looped và Priority

### Looped Property

```lua
-- Set Looped trước khi Play
track.Looped = true   -- Animation sẽ lặp vô hạn
track.Looped = false  -- Animation chỉ phát 1 lần

track:Play()
```

### Khi nào dùng Looped?

```
┌─────────────────────────────────────────────────────────────────────────┐
│                    LOOPED USAGE                                          │
├─────────────────────────────────────────────────────────────────────────┤
│                                                                         │
│  LOOPED = TRUE:                                                         │
│  ├── Walk/Run animations                                                │
│  ├── Idle animations                                                    │
│  ├── Swimming                                                           │
│  ├── Dancing (liên tục)                                                 │
│  └── Any continuous action                                              │
│                                                                         │
│  LOOPED = FALSE:                                                        │
│  ├── Attack animations                                                  │
│  ├── Jump (một lần)                                                     │
│  ├── Emotes (wave, cheer)                                               │
│  ├── Death animation                                                    │
│  └── Any one-shot action                                                │
│                                                                         │
└─────────────────────────────────────────────────────────────────────────┘
```

### Priority Property

```lua
-- Set Priority trước khi Play
track.Priority = Enum.AnimationPriority.Idle
track.Priority = Enum.AnimationPriority.Movement
track.Priority = Enum.AnimationPriority.Action
track.Priority = Enum.AnimationPriority.Action2
track.Priority = Enum.AnimationPriority.Action3
track.Priority = Enum.AnimationPriority.Action4

track:Play()
```

### Priority Hierarchy

```
┌─────────────────────────────────────────────────────────────────────────┐
│                    PRIORITY HIERARCHY                                    │
├─────────────────────────────────────────────────────────────────────────┤
│                                                                         │
│  THẤP ─────────────────────────────────────────────────────────► CAO    │
│                                                                         │
│  Core → Idle → Movement → Action → Action2 → Action3 → Action4          │
│   0       1        2         3         4         5         6            │
│                                                                         │
│  Animation priority CAO sẽ ĐÈ LÊN priority THẤP                         │
│                                                                         │
│  VÍ DỤ:                                                                 │
│  • Walk (Movement) đang chạy                                            │
│  • Attack (Action) được Play                                            │
│  • → Attack sẽ đè lên Walk ở các body parts chung                       │
│                                                                         │
└─────────────────────────────────────────────────────────────────────────┘
```

### Ví dụ: Setup Animation System

```lua
-- Idle animation - luôn chạy nền
local idleTrack = animator:LoadAnimation(idleAnim)
idleTrack.Priority = Enum.AnimationPriority.Idle
idleTrack.Looped = true
idleTrack:Play()

-- Walk animation - khi di chuyển
local walkTrack = animator:LoadAnimation(walkAnim)
walkTrack.Priority = Enum.AnimationPriority.Movement
walkTrack.Looped = true

-- Attack animation - khi tấn công
local attackTrack = animator:LoadAnimation(attackAnim)
attackTrack.Priority = Enum.AnimationPriority.Action
attackTrack.Looped = false  -- Chỉ đánh 1 lần

-- Khi di chuyển
humanoid.Running:Connect(function(speed)
    if speed > 0.1 then
        walkTrack:Play()
    else
        walkTrack:Stop()
    end
end)

-- Khi tấn công
local function attack()
    attackTrack:Play()  -- Sẽ đè lên walk ở phần tay
end
```

## 2.5 Xử lý Sự kiện Stopped

### Các Events quan trọng

```lua
-- Event Stopped: fire khi animation dừng
track.Stopped:Connect(function()
    print("Animation đã dừng!")
end)

-- Event Ended: fire khi animation kết thúc tự nhiên (non-looped)
track.Ended:Connect(function()
    print("Animation kết thúc!")
end)

-- Event DidLoop: fire mỗi khi animation lặp lại
track.DidLoop:Connect(function()
    print("Animation vừa loop!")
end)
```

### Sự khác nhau giữa Stopped và Ended

```
┌─────────────────────────────────────────────────────────────────────────┐
│                    STOPPED VS ENDED                                      │
├─────────────────────────────────────────────────────────────────────────┤
│                                                                         │
│  STOPPED:                                                               │
│  ├── Fire khi gọi track:Stop()                                          │
│  ├── Fire khi animation kết thúc tự nhiên (non-looped)                  │
│  └── Bao gồm cả Ended                                                   │
│                                                                         │
│  ENDED:                                                                 │
│  ├── CHỈ fire khi animation kết thúc tự nhiên                           │
│  ├── KHÔNG fire khi gọi Stop() thủ công                                 │
│  └── Chỉ cho non-looped animations                                      │
│                                                                         │
│  LOOPED ANIMATIONS:                                                     │
│  ├── Stopped: fire khi gọi Stop()                                       │
│  ├── Ended: KHÔNG BAO GIỜ fire (vì không bao giờ end)                   │
│  └── DidLoop: fire mỗi lần lặp                                          │
│                                                                         │
└─────────────────────────────────────────────────────────────────────────┘
```

### Ví dụ: Combo System với Stopped

```lua
local comboIndex = 0
local comboTracks = {
    animator:LoadAnimation(attack1Anim),
    animator:LoadAnimation(attack2Anim),
    animator:LoadAnimation(attack3Anim),
}

local canCombo = false
local comboResetTime = 1  -- 1 giây để reset combo

-- Setup events cho mỗi attack
for i, track in ipairs(comboTracks) do
    track.Priority = Enum.AnimationPriority.Action
    track.Looped = false
    
    -- Khi animation kết thúc
    track.Stopped:Connect(function()
        canCombo = false
        
        -- Reset combo sau một khoảng thời gian
        task.delay(comboResetTime, function()
            if not canCombo then
                comboIndex = 0
            end
        end)
    end)
end

local function attack()
    -- Tăng combo index
    comboIndex = comboIndex + 1
    if comboIndex > #comboTracks then
        comboIndex = 1
    end
    
    -- Phát animation tương ứng
    local track = comboTracks[comboIndex]
    track:Play(0)
    canCombo = true
end
```

### Ví dụ: Chờ Animation kết thúc

```lua
-- Hàm phát animation và chờ kết thúc
local function playAndWait(track)
    track:Play()
    track.Stopped:Wait()  -- Chờ cho đến khi dừng
end

-- Sử dụng
playAndWait(introTrack)
print("Intro xong!")
playAndWait(attackTrack)
print("Attack xong!")
playAndWait(victoryTrack)
print("Victory xong!")
```

---

# 📜 PHẦN 3: SCRIPT MẪU ĐẦU TIÊN

---

## 3.1 Script Cơ Bản Nhất

### LocalScript trong StarterPlayerScripts

```lua
--[[
    Script: PlayBasicAnimation.lua
    Vị trí: StarterPlayer > StarterPlayerScripts
    Mục đích: Phát một animation khi game bắt đầu
--]]

local Players = game:GetService("Players")

-- Lấy player và character
local player = Players.LocalPlayer
local character = player.Character or player.CharacterAdded:Wait()

-- Lấy Humanoid và Animator
local humanoid = character:WaitForChild("Humanoid")
local animator = humanoid:WaitForChild("Animator")

-- Tạo Animation object
local animation = Instance.new("Animation")
animation.AnimationId = "rbxassetid://507770239"  -- Wave animation mẫu

-- Load animation
local track = animator:LoadAnimation(animation)

-- Phát animation
track:Play()

print("Animation đang phát!")
```

### Giải thích từng dòng

```
┌─────────────────────────────────────────────────────────────────────────┐
│                    GIẢI THÍCH CODE                                       │
├─────────────────────────────────────────────────────────────────────────┤
│                                                                         │
│  local Players = game:GetService("Players")                             │
│  └── Lấy service Players để truy cập LocalPlayer                        │
│                                                                         │
│  local player = Players.LocalPlayer                                     │
│  └── Lấy player đang chạy script này                                    │
│                                                                         │
│  player.Character or player.CharacterAdded:Wait()                       │
│  └── Lấy character, nếu chưa có thì chờ                                 │
│                                                                         │
│  character:WaitForChild("Humanoid")                                     │
│  └── Chờ Humanoid load xong (an toàn hơn FindFirstChild)                │
│                                                                         │
│  humanoid:WaitForChild("Animator")                                      │
│  └── Chờ Animator có sẵn trong Humanoid                                 │
│                                                                         │
│  Instance.new("Animation")                                              │
│  └── Tạo Animation object mới                                           │
│                                                                         │
│  animator:LoadAnimation(animation)                                      │
│  └── Load animation vào bộ nhớ, trả về AnimationTrack                   │
│                                                                         │
│  track:Play()                                                           │
│  └── Phát animation!                                                    │
│                                                                         │
└─────────────────────────────────────────────────────────────────────────┘
```

## 3.2 Script Phát Animation Khi Nhấn Phím

```lua
--[[
    Script: PlayAnimationOnKeyPress.lua
    Vị trí: StarterPlayer > StarterPlayerScripts
    Mục đích: Phát animation khi nhấn phím E
--]]

local Players = game:GetService("Players")
local UserInputService = game:GetService("UserInputService")

-- Setup
local player = Players.LocalPlayer
local character = player.Character or player.CharacterAdded:Wait()
local humanoid = character:WaitForChild("Humanoid")
local animator = humanoid:WaitForChild("Animator")

-- Tạo và load animation
local waveAnim = Instance.new("Animation")
waveAnim.AnimationId = "rbxassetid://507770239"  -- Wave

local waveTrack = animator:LoadAnimation(waveAnim)
waveTrack.Priority = Enum.AnimationPriority.Action
waveTrack.Looped = false

-- Biến để tránh spam
local canPlay = true

-- Xử lý input
UserInputService.InputBegan:Connect(function(input, gameProcessed)
    -- Bỏ qua nếu đang chat hoặc trong GUI
    if gameProcessed then return end
    
    -- Kiểm tra phím E
    if input.KeyCode == Enum.KeyCode.E then
        if canPlay then
            canPlay = false
            waveTrack:Play()
            print("Wave!")
            
            -- Cho phép play lại sau khi animation dừng
            waveTrack.Stopped:Wait()
            canPlay = true
        end
    end
end)

print("Nhấn E để vẫy tay!")
```

## 3.3 Script Hệ Thống Animation Đầy Đủ

```lua
--[[
    Script: AnimationSystem.lua
    Vị trí: StarterPlayer > StarterPlayerScripts
    Mục đích: Hệ thống animation hoàn chỉnh với nhiều animations
--]]

local Players = game:GetService("Players")
local UserInputService = game:GetService("UserInputService")

-- Animation IDs (thay bằng ID của bạn)
local ANIMATION_IDS = {
    Wave = "rbxassetid://507770239",
    Cheer = "rbxassetid://507770677",
    Point = "rbxassetid://507770453",
    Dance = "rbxassetid://507771019",
}

-- Keybinds
local KEYBINDS = {
    [Enum.KeyCode.One] = "Wave",
    [Enum.KeyCode.Two] = "Cheer",
    [Enum.KeyCode.Three] = "Point",
    [Enum.KeyCode.Four] = "Dance",
}

-- Variables
local player = Players.LocalPlayer
local animationTracks = {}
local currentTrack = nil

-- Function: Setup animations cho character
local function setupAnimations(character)
    local humanoid = character:WaitForChild("Humanoid")
    local animator = humanoid:WaitForChild("Animator")
    
    -- Load tất cả animations
    animationTracks = {}
    for name, id in pairs(ANIMATION_IDS) do
        local anim = Instance.new("Animation")
        anim.AnimationId = id
        anim.Name = name
        
        local track = animator:LoadAnimation(anim)
        track.Priority = Enum.AnimationPriority.Action
        
        -- Dance thì loop, còn lại thì không
        track.Looped = (name == "Dance")
        
        animationTracks[name] = track
    end
    
    print("Loaded", #animationTracks, "animations!")
end

-- Function: Phát animation
local function playAnimation(name)
    local track = animationTracks[name]
    if not track then
        warn("Animation not found:", name)
        return
    end
    
    -- Dừng animation hiện tại nếu có
    if currentTrack and currentTrack.IsPlaying then
        currentTrack:Stop(0.1)
    end
    
    -- Phát animation mới
    track:Play(0.1)
    currentTrack = track
    
    print("Playing:", name)
end

-- Function: Dừng animation hiện tại
local function stopCurrentAnimation()
    if currentTrack and currentTrack.IsPlaying then
        currentTrack:Stop(0.2)
        currentTrack = nil
        print("Stopped animation")
    end
end

-- Setup khi character spawn
local function onCharacterAdded(character)
    setupAnimations(character)
end

-- Connect events
player.CharacterAdded:Connect(onCharacterAdded)

-- Setup cho character hiện tại (nếu đã có)
if player.Character then
    onCharacterAdded(player.Character)
end

-- Input handling
UserInputService.InputBegan:Connect(function(input, gameProcessed)
    if gameProcessed then return end
    
    -- Kiểm tra keybind
    local animName = KEYBINDS[input.KeyCode]
    if animName then
        playAnimation(animName)
    end
    
    -- Phím Q để dừng
    if input.KeyCode == Enum.KeyCode.Q then
        stopCurrentAnimation()
    end
end)

-- Print hướng dẫn
print("=== ANIMATION SYSTEM ===")
print("Press 1: Wave")
print("Press 2: Cheer")
print("Press 3: Point")
print("Press 4: Dance")
print("Press Q: Stop")
print("========================")
```

## 3.4 Script với Animation Events

```lua
--[[
    Script: AnimationWithEvents.lua
    Vị trí: StarterPlayer > StarterPlayerScripts
    Mục đích: Sử dụng Animation Events và Markers
--]]

local Players = game:GetService("Players")
local UserInputService = game:GetService("UserInputService")

local player = Players.LocalPlayer
local character = player.Character or player.CharacterAdded:Wait()
local humanoid = character:WaitForChild("Humanoid")
local animator = humanoid:WaitForChild("Animator")

-- Load animation
local attackAnim = Instance.new("Animation")
attackAnim.AnimationId = "rbxassetid://YOUR_ATTACK_ANIMATION_ID"

local attackTrack = animator:LoadAnimation(attackAnim)
attackTrack.Priority = Enum.AnimationPriority.Action
attackTrack.Looped = false

--[[
    ANIMATION EVENTS
    Giả sử animation có marker tên "HitPoint" ở giữa animation
    (Marker được thêm trong Animation Editor)
--]]

-- Event: Khi đến marker "HitPoint"
attackTrack:GetMarkerReachedSignal("HitPoint"):Connect(function()
    print("HIT! Deal damage here!")
    
    -- Ví dụ: Tạo hitbox, deal damage
    -- dealDamage()
    -- playSound("Slash")
    -- spawnVFX("SlashEffect")
end)

-- Event: Khi animation bắt đầu
attackTrack:GetPropertyChangedSignal("IsPlaying"):Connect(function()
    if attackTrack.IsPlaying then
        print("Attack started!")
    end
end)

-- Event: Khi animation kết thúc
attackTrack.Stopped:Connect(function()
    print("Attack finished!")
end)

-- Event: Mỗi khi animation loop (nếu Looped = true)
attackTrack.DidLoop:Connect(function()
    print("Animation looped!")
end)

-- Input: Nhấn F để attack
UserInputService.InputBegan:Connect(function(input, gameProcessed)
    if gameProcessed then return end
    
    if input.KeyCode == Enum.KeyCode.F then
        if not attackTrack.IsPlaying then
            attackTrack:Play()
        end
    end
end)

print("Nhấn F để attack!")
```

### Cách thêm Marker trong Animation Editor

```
┌─────────────────────────────────────────────────────────────────────────┐
│                    THÊM MARKER TRONG ANIMATION EDITOR                    │
├─────────────────────────────────────────────────────────────────────────┤
│                                                                         │
│  1. Mở Animation Editor với animation của bạn                           │
│  2. Di chuyển Playhead đến thời điểm muốn thêm marker                   │
│  3. Right-click vào Timeline                                            │
│  4. Chọn "Add Marker"                                                   │
│  5. Đặt tên marker (ví dụ: "HitPoint", "FootStep", "VFX")               │
│  6. Publish animation                                                   │
│                                                                         │
│  Timeline:                                                              │
│  0:00    0:05    0:10    0:15    0:20                                   │
│  │       │       │       │       │                                      │
│  ●───────────────▲───────────────●                                      │
│                  │                                                      │
│            Marker "HitPoint"                                            │
│                                                                         │
└─────────────────────────────────────────────────────────────────────────┘
```

---

# 📝 TỔNG KẾT

---

## Checklist kiến thức đã học

### Phần 1: Các Class Quan Trọng
- [x] Animation - chứa AnimationId
- [x] Animator - load và quản lý animations
- [x] AnimationTrack - điều khiển Play/Stop

### Phần 2: Kỹ Năng Cần Có
- [x] Tạo Animation object
- [x] Load animation với Animator
- [x] Play và Stop animation
- [x] Thiết lập Looped và Priority
- [x] Xử lý sự kiện Stopped/Ended

### Phần 3: Script Mẫu
- [x] Script cơ bản nhất
- [x] Script phát animation khi nhấn phím
- [x] Hệ thống animation đầy đủ
- [x] Animation với Events và Markers

---

## Quick Reference

```lua
-- TEMPLATE CƠ BẢN
local character = player.Character or player.CharacterAdded:Wait()
local humanoid = character:WaitForChild("Humanoid")
local animator = humanoid:WaitForChild("Animator")

local animation = Instance.new("Animation")
animation.AnimationId = "rbxassetid://YOUR_ID"

local track = animator:LoadAnimation(animation)
track.Priority = Enum.AnimationPriority.Action
track.Looped = false

track:Play(0.1)   -- Phát với fade 0.1s
track:Stop(0.2)   -- Dừng với fade 0.2s

track.Stopped:Connect(function()
    print("Done!")
end)
```

---

## Bước tiếp theo

Sau khi nắm vững Scripting Animation Cơ Bản:

1. **1.5 Project Thực Hành** - Áp dụng vào Wave Emote, Jump Animation, Idle Variations
2. **Level 2** - Animation Nâng Cao: Blending, Speed, Markers, State Machine

---

## Tài liệu tham khảo

- [Animation Class](https://create.roblox.com/docs/reference/engine/classes/Animation)
- [Animator Class](https://create.roblox.com/docs/reference/engine/classes/Animator)
- [AnimationTrack Class](https://create.roblox.com/docs/reference/engine/classes/AnimationTrack)
- [Animation Priority](https://create.roblox.com/docs/reference/engine/enums/AnimationPriority)